# Import and save all data sets to pandas

### Imports and function defs

In [1]:
import pandas as pd
import numpy as np

import csv,datetime,time
import psycopg2
import sys

import gm2
import trfp
import analysis_helper as helper

def get_subrun_df(start_run, end_run):
    
    dsn  = "dbname=gm2_online_prod user=gm2_reader host=g2db-priv port=5433"
    conn = psycopg2.connect(dsn)
    curr = conn.cursor()
    
    # get times by subrun
    subrun_time_columns = ['run', 'subrun', 'start_time', 'end_time', 'start_gps', 'end_gps']
    sql = "select "+", ".join(subrun_time_columns)+" from gm2dq.subrun_time where run >= %i and run <= %i order by run, subrun" % (start_run, end_run)
    curr.execute(sql)
    conn.commit()
    subrun_time = curr.fetchall()
    subrun_time_df = pd.DataFrame(subrun_time, columns=subrun_time_columns)
    
    # get ctags by subruns
    ctagswithdqc_columns = ['run', 'subrun', 'ctags', 't0val', 'fills']
    sql = "select "+", ".join(ctagswithdqc_columns)+" from gm2dq.ctagswithdqc where run >= %i and run <= %i order by run, subrun" % (start_run, end_run)
    curr.execute(sql)
    conn.commit()
    ctagswithdqc = curr.fetchall()
    ctagswithdqc_df = pd.DataFrame(ctagswithdqc, columns=ctagswithdqc_columns)
    
    # get subrun status database
    subrun_status_columns = ['run', 'subrun', 'quad_condition', 'kicker_condition', 'quad_ok',
                             'ctags_ok', 'losses_ok', 'fillcuts_ok', 'field_ok', 'trolley_period', 'field_period',
                             'ctags_loose_ok', 'quad_loose_ok', 'ctags_repeat_ok', 'losses_repeat_ok', 'fillcuts_repeat_ok']
    sql = "select "+", ".join(subrun_status_columns)+" from gm2dq.subrun_status where run >= %i and run <= %i order by run, subrun" % (start_run, end_run)
    curr.execute(sql)
    conn.commit()
    subrun_status = curr.fetchall()
    subrun_status_df = pd.DataFrame(subrun_status, columns=subrun_status_columns)
    
    # merge times, ctags, status into one subrun dataframe
    subrun_df = pd.merge(subrun_time_df, ctagswithdqc_df)
    subrun_df = pd.merge(subrun_df, subrun_status_df)
    subrun_df['ok'] = subrun_df['quad_ok'] & subrun_df['ctags_ok'] & subrun_df['losses_ok'] & subrun_df['fillcuts_ok'] & subrun_df['field_ok']
    subrun_df['start_time'] = subrun_df['start_time'].astype(np.int64)/1e9 + 5*60*60
    subrun_df['end_time'] = subrun_df['end_time'].astype(np.int64)/1e9 + 5*60*60
    subrun_df['start_gps'] = subrun_df['start_gps'].astype(np.int64)/1.0e9 + 5*60*60
    subrun_df['end_gps'] = subrun_df['end_gps'].astype(np.int64)/1.0e9 + 5*60*60
    
    print subrun_df.shape
    
    return subrun_df

Welcome to JupyROOT 6.12/04


## 60 hr

In [5]:
%%time

prefix = 'data1/newg2/DataProduction/Offline/ArtTFSDir/v9_21_05_dev/FieldPlainRootOutput_'

tr_interp_df_1 = helper.root_to_pandas([3956], prefix=prefix, tr_run=True)
fp_interp_df_1 = helper.root_to_pandas(range(3959,3995), prefix=prefix, tr_run=False)
tr_interp_df_2 = helper.root_to_pandas([3997], prefix=prefix, tr_run=True)


Reading event 8780/8781 99.99%           
loop done: 100%                                    
Reading event 2630/2634 99.85% 
loop done: 100%                                    
Reading event 154050/154052 100.00%                                                                                                         
loop done: 100%                                    
Reading event 8730/8735 99.94%          
loop done: 100%                                    
Reading event 2610/2619 99.66%  
loop done: 100%                                    
CPU times: user 57.3 s, sys: 12.3 s, total: 1min 9s
Wall time: 2min 9s


In [3]:
%%time

start_run = 15921
end_run = 15992

subrun_df = get_subrun_df(start_run, end_run)

(27275, 24)
CPU times: user 221 ms, sys: 34 ms, total: 255 ms
Wall time: 1.29 s


In [6]:
%%time

# Save all the dataframes in one hdf5 file

filename = 'hdf5/60hr.h5'

tr_interp_df_1.to_hdf(filename, key='tr_df_1')
fp_interp_df_1.to_hdf(filename, key='fp_df_1')
tr_interp_df_2.to_hdf(filename, key='tr_df_2')

subrun_df.to_hdf(filename, key='subrun_df')

CPU times: user 59 ms, sys: 393 ms, total: 452 ms
Wall time: 457 ms


## 9 day

In [ ]:
%%time

prefix = 'data1/newg2/DataProduction/Offline/ArtTFSDir/v9_21_05_dev/FieldPlainRootOutput_'

tr_interp_df_1 = helper.root_to_pandas([4138], prefix=prefix, tr_run=True)
fp_interp_df_1 = helper.root_to_pandas(range(4141,4179), prefix=prefix, tr_run=False)
tr_interp_df_2 = helper.root_to_pandas([4181], prefix=prefix, tr_run=True)

## MAGNET CYCLE

tr_interp_df_3 = helper.root_to_pandas([4189], prefix=prefix, tr_run=True)
fp_interp_df_2 = helper.root_to_pandas(range(4193,4223), prefix=prefix, tr_run=False)
tr_interp_df_4 = helper.root_to_pandas([4226], prefix=prefix, tr_run=True)
fp_interp_df_3 = helper.root_to_pandas(range(4229,4263), prefix=prefix, tr_run=False)
tr_interp_df_5 = helper.root_to_pandas([4265], prefix=prefix, tr_run=True)
fp_interp_df_4 = helper.root_to_pandas(range(4283,4489), prefix=prefix, tr_run=False)
tr_interp_df_6 = helper.root_to_pandas([4493], prefix=prefix, tr_run=True)

In [ ]:
%%time

start_run = 16355
end_run = 16539

subrun_df = get_subrun_df(start_run, end_run)

In [ ]:
%%time

## Save all the dataframes in one hdf5 file

filename = 'hdf5/9day.h5'

tr_interp_df_1.to_hdf(filename, key='tr_df_1')
fp_interp_df_1.to_hdf(filename, key='fp_df_1')
tr_interp_df_2.to_hdf(filename, key='tr_df_2')

tr_interp_df_3.to_hdf(filename, key='tr_df_3')
fp_interp_df_2.to_hdf(filename, key='fp_df_2')
tr_interp_df_4.to_hdf(filename, key='tr_df_4')
fp_interp_df_3.to_hdf(filename, key='fp_df_3')
tr_interp_df_5.to_hdf(filename, key='tr_df_5')
fp_interp_df_4.to_hdf(filename, key='fp_df_4')
tr_interp_df_6.to_hdf(filename, key='tr_df_6')

subrun_df.to_hdf(filename, key='subrun_df')

## Endgame

In [ ]:
%%time

prefix = 'data1/newg2/DataProduction/Offline/ArtTFSDir/v9_21_05_dev/FieldPlainRootOutput_'

# tr_interp_df_1 = helper.root_to_pandas([4997], prefix=prefix, tr_run=True)
# fp_interp_df_1 = helper.root_to_pandas(range(5000,5050), prefix=prefix, tr_run=False)
tr_interp_df_2 = helper.root_to_pandas([5054], prefix=prefix, tr_run=True)
fp_interp_df_2 = helper.root_to_pandas(range(5057,5101), prefix=prefix, tr_run=False)
tr_interp_df_3 = helper.root_to_pandas([5103], prefix=prefix, tr_run=True)

## MAGNET CYCLE runs 5108-5114
# magnet_cycle_df_1 = helper.root_to_pandas(range(5108,5115), prefix=prefix, tr_run=False)

tr_interp_df_4 = helper.root_to_pandas([5117], prefix=prefix, tr_run=True)
fp_interp_df_3 = helper.root_to_pandas(range(5120,5155), prefix=prefix, tr_run=False)
tr_interp_df_5 = helper.root_to_pandas([5157], prefix=prefix, tr_run=True)

## MAGNET CYCLE runs 5160-5166
# magnet_cycle_df_2 = helper.root_to_pandas(range(5160,5167), prefix=prefix, tr_run=False)

tr_interp_df_6 = helper.root_to_pandas([5169], prefix=prefix, tr_run=True)
fp_interp_df_4 = helper.root_to_pandas(range(5172,5215), prefix=prefix, tr_run=False)
tr_interp_df_7 = helper.root_to_pandas([5217], prefix=prefix, tr_run=True)
fp_interp_df_5 = helper.root_to_pandas(range(5220,5257), prefix=prefix, tr_run=False)
tr_interp_df_8 = helper.root_to_pandas([5259], prefix=prefix, tr_run=True)
fp_interp_df_6 = helper.root_to_pandas(range(5262,5301), prefix=prefix, tr_run=False)
tr_interp_df_9 = helper.root_to_pandas([5303], prefix=prefix, tr_run=True)


In [ ]:
%%time

start_run = 16908
end_run = 17528

subrun_df = get_subrun_df(start_run, end_run)

In [ ]:
%%time

## Save all the dataframes in one hdf5 file

filename = 'hdf5/endgame.h5'

subrun_df.to_hdf(filename, key='subrun_df')

# tr_interp_df_1.to_hdf(filename, key='tr_df_1')
# fp_interp_df_1.to_hdf(filename, key='fp_df_1')
tr_interp_df_2.to_hdf(filename, key='tr_df_2')
fp_interp_df_2.to_hdf(filename, key='fp_df_2')
tr_interp_df_3.to_hdf(filename, key='tr_df_3')

# ## MAGNET CYCLE runs 5108-5114
# magnet_cycle_df_1.to_hdf(filename, key='magnet_cycle_df_1')

tr_interp_df_4.to_hdf(filename, key='tr_df_4')
fp_interp_df_3.to_hdf(filename, key='fp_df_3')
tr_interp_df_5.to_hdf(filename, key='tr_df_5')

## MAGNET CYCLE runs 5160-5166
# magnet_cycle_df_2.to_hdf(filename, key='magnet_cycle_df_2')

tr_interp_df_6.to_hdf(filename, key='tr_df_6')
fp_interp_df_4.to_hdf(filename, key='fp_df_4')
tr_interp_df_7.to_hdf(filename, key='tr_df_7')
fp_interp_df_5.to_hdf(filename, key='fp_df_5')
tr_interp_df_8.to_hdf(filename, key='tr_df_8')
fp_interp_df_6.to_hdf(filename, key='fp_df_6')
tr_interp_df_9.to_hdf(filename, key='tr_df_9')

## High Kick

Notes: No run 4122 exists

In [ ]:
%%time

prefix = 'data1/newg2/DataProduction/Offline/ArtTFSDir/v9_21_05_dev/FieldPlainRootOutput_'

tr_interp_df_1 = helper.root_to_pandas([4058], prefix=prefix, tr_run=True)
fp_interp_df_1 = helper.root_to_pandas(range(4061,4096), prefix=prefix, tr_run=False)
tr_interp_df_2 = helper.root_to_pandas([4098], prefix=prefix, tr_run=True)
fp_interp_df_2 = helper.root_to_pandas(range(4101,4122)+range(4123,4136), prefix=prefix, tr_run=False)
tr_interp_df_3 = helper.root_to_pandas([4138], prefix=prefix, tr_run=True)


In [ ]:
%%time

start_run = 16110
end_run = 16256

subrun_df = get_subrun_df(start_run, end_run)

In [ ]:
%%time

# # Save all the dataframes in one hdf5 file

filename = 'hdf5/highkick.h5'

tr_interp_df_1.to_hdf(filename, key='tr_df_1')
fp_interp_df_1.to_hdf(filename, key='fp_df_1')
tr_interp_df_2.to_hdf(filename, key='tr_df_2')
fp_interp_df_2.to_hdf(filename, key='fp_df_2')
tr_interp_df_3.to_hdf(filename, key='tr_df_3')

subrun_df.to_hdf(filename, key='subrun_df')

## Low Kick

Something is wrong with the omega_a database on this one. No GPS times, for instance?

In [4]:
%%time

prefix = 'data1/newg2/DataProduction/Offline/ArtTFSDir/v9_21_05_dev/FieldPlainRootOutput_'

tr_interp_df_1 = helper.root_to_pandas([4539], prefix=prefix, tr_run=True)
# fp_interp_df_1 = helper.root_to_pandas(range(4542,4582), prefix=prefix, tr_run=False)
tr_interp_df_2 = helper.root_to_pandas([4584], prefix=prefix, tr_run=True)


Reading event 8780/8789 99.90%                
loop done: 100%                                    
Reading event 2630/2636 99.77%    
loop done: 100%                                    
Reading event 8680/8683 99.97%      
loop done: 100%                                    
Reading event 2600/2604 99.85%   
loop done: 100%                                    
CPU times: user 4.18 s, sys: 406 ms, total: 4.58 s
Wall time: 11.4 s


In [ ]:
start_run = 16669
end_run = 16714

subrun_df = get_subrun_df(start_run, end_run)

In [ ]:
%%time

## Save all the dataframes in one hdf5 file

filename = 'hdf5/lowkick.h5'

tr_interp_df_1.to_hdf(filename, key='tr_df_1')
# fp_interp_df_1.to_hdf(filename, key='fp_df_1')
tr_interp_df_2.to_hdf(filename, key='tr_df_2')

# subrun_df.to_hdf(filename, key='subrun_df')